# Game Optimization Project

In [1]:
from src.utils import TicTacToe, ConnectFour

In [2]:
game1 = TicTacToe()

In [3]:
game1.get_game().model_dump()

{'id': UUID('7e9de6b2-47d3-449d-8c9a-42253bf08733'),
 'createdat': datetime.datetime(2024, 12, 5, 7, 10, 48, 883678),
 'winner': None,
 'game_type': <GameType.TIC_TAC_TOE: 'TicTacToe'>,
 'moves': []}

In [4]:
game2 = ConnectFour()

In [5]:
game2.get_game().model_dump()

{'id': UUID('f82abc64-d6df-4ce8-b290-f966366d08e7'),
 'createdat': datetime.datetime(2024, 12, 5, 7, 10, 48, 883678),
 'winner': None,
 'game_type': <GameType.CONNECT_4: 'Connect4'>,
 'moves': []}